In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

from evaluation import *
from data_loader import *
import random
from datetime import datetime

In [2]:
colors = {
    "DQN": 'darkorange',
    "PPO": 'forestgreen',
    "CMA": 'cyan',
    "ODT": 'blueviolet'
}

In [3]:
starting_exp = 4072
ending_exp = 4107
save_processed_data = True
load_agent = True
load_station = True
exp_lists_preformatted = ['4000_4035.csv', '4072_4107.csv'] # Existing formatted datasets
#exp_lists_preformatted = []

target_agg_count = 6
target_reward_type = 'communal'

exp_agent_data = []
exp_station_data = []

experiments = range(starting_exp, ending_exp + 1)

# REMOVE THIS
#algos = ['DQN', 'PPO', 'CMA', 'ODT']

if len(exp_lists_preformatted) == 0:
    for ind, exp_num in enumerate(experiments):
        config_fname = f'./Exp_{exp_num}/config.yaml'
        
        c = load_config_file(config_fname)
        nn_c = c['nn_hyperparameters']
        federated_c = c['federated_learning_settings']
        algo_c = c['algorithm_settings']
        env_c = c['environment_settings']
        eval_c = c['eval_config']
        cma_c = c['cma_parameters']
    
        if federated_c['aggregation_count'] != target_agg_count:
            print(f"Experiment {exp_num} does not have matching aggregation level")
            continue

        rtype = 'communal' if nn_c['average_rewards_when_training'] else 'greedy'

        if rtype != target_reward_type:
            print(f"Experiment {exp_num} does not have matching reward type")
            continue
        
        ev_info = []
    
        seed = env_c['seed']
    
        algorithm_dm = algo_c['algorithm']
        
        def load_from_json_with_error_handling(filepath, columns_specific):
            try:
                return read_csv_data(filepath, columns=columns_specific)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON from {filepath}: {e.msg} at line {e.lineno}, column {e.colno}")
                return None  # Handle the error and return None or an empty object
        
        
        d_base = f"../../../../storage_1/metrics/Exp_{exp_num}"
        
        if not os.path.exists(d_base):
            d_base = f"../metrics/Exp_{exp_num}"
                
        base_path = f"{d_base}/train/metrics"
    
        print(f'Loading {base_path}....csv')
        if load_agent:
            agent_data = load_from_json_with_error_handling(f'{base_path}_agent_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'reward', 'agent_index', 'distance', 'average_battery'])
            agent_data['seed'] = seed
            agent_data['exp_num'] = exp_num
            agent_data['algorithm'] = algorithm_dm
            #agent_data['algorithm'] = algos[ind % len(algos)] # REMOVE THIS AND UNCOMMENT PREVIOUS LINE!
            agent_data['season'] = env_c['season']
            exp_agent_data.append(agent_data)
        if load_station:
            station_data = load_from_json_with_error_handling(f'{base_path}_station_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'station_index', 'traffic'])
            station_data['seed'] = seed        
            station_data['exp_num'] = exp_num     
            station_data['algorithm'] = algorithm_dm
            #station_data['algorithm'] = algos[ind % len(algos)] # REMOVE THIS AND UNCOMMENT PREVIOUS LINE!
            station_data['season'] = env_c['season']
            exp_station_data.append(station_data)

In [4]:
if len(exp_lists_preformatted) == 0:
    # Convert data to DataFrame for easier manipulation
    if load_agent:
        df_agent = pd.concat(exp_agent_data, ignore_index=True)

        cumulative_agent_df = (
            df_agent
            .groupby(
                ['episode', 'zone', 'aggregation', 'agent_index', 'seed', 'exp_num', 'algorithm', 'season']
            )[['distance', 'average_battery', 'reward']]
            .sum()
            .reset_index()
        )

        # Rename the 'reward' column to 'cumulative_reward' for clarity
        cumulative_agent_df.rename(columns={'average_battery': 'battery_used'}, inplace=True)
        cumulative_agent_df.rename(columns={'distance': 'distance_traveled'}, inplace=True)
        
        cumulative_agent_df['episode'] = cumulative_agent_df['aggregation'] * (6000/target_agg_count) + cumulative_agent_df['episode']
    
    if load_station:
        df_station = pd.concat(exp_station_data, ignore_index=True)
    
        print('Cumulating data')    
        cumulative_station_df = (
            df_station
            .groupby(
                ['episode', 'zone', 'aggregation', 'station_index', 'seed', 'exp_num', 'algorithm', 'season']
            )[['traffic']]
            .max()
            .reset_index()
        )

        print('Recalculating episode')    
        # Get recalculated episodes using (aggregation number * episodes per aggregation) + episode number
        cumulative_station_df['episode'] = cumulative_station_df['aggregation'] * (6000/target_agg_count)  + cumulative_station_df['episode']

In [5]:
# Define the file path with starting_exp and ending_exp variables
file_path_for_processed_data = f'../../../../storage_1/metrics/formatted_experiment_data/part_4/'

if len(exp_lists_preformatted) == 0:

    # Get the data for the last episode only
    if load_agent:
        cumulative_agent_df = cumulative_agent_df[cumulative_agent_df['episode'] == cumulative_agent_df['episode'].max()]
    
    if load_station:
        cumulative_station_df = cumulative_station_df[cumulative_station_df['episode'] == cumulative_station_df['episode'].max()]

    
    if save_processed_data:
        if not os.path.exists(file_path_for_processed_data):
            os.makedirs(file_path_for_processed_data)
        
        # Save the DataFrame to a CSV file
        if load_agent:
            cumulative_agent_df.to_csv(f"{file_path_for_processed_data}/agent_{starting_exp}_{ending_exp}.csv", index=False)
            print(f"Data successfully saved to {file_path_for_processed_data}/agent_{starting_exp}_{ending_exp}.csv")

        if load_station:
            cumulative_station_df.to_csv(f"{file_path_for_processed_data}/station_{starting_exp}_{ending_exp}.csv", index=False)
            print(f"Data successfully saved to {file_path_for_processed_data}/station_{starting_exp}_{ending_exp}.csv")

if len(exp_lists_preformatted) > 0:
    if load_agent:
        agent_dataframes = []
        
        # Load and combine datasets
        for existing_set in exp_lists_preformatted:
            loaded_dataset = pd.read_csv(f"{file_path_for_processed_data}/agent_{existing_set}")
            # Combine datasets
            agent_dataframes.append(loaded_dataset)
    
        cumulative_agent_df = pd.concat(agent_dataframes, ignore_index=True)

    if load_station:
        station_dataframes = []
        
        # Load and combine datasets
        for existing_set in exp_lists_preformatted:
            loaded_dataset = pd.read_csv(f"{file_path_for_processed_data}/station_{existing_set}")
            # Combine datasets
            station_dataframes.append(loaded_dataset)
    
        cumulative_station_df = pd.concat(station_dataframes, ignore_index=True)

In [9]:
if load_agent:
    algos = cumulative_agent_df['algorithm'].unique()
if load_station:
    algos = cumulative_station_df['algorithm'].unique()

for algo in algos:
    print(f"===== Algorithm: {algo} =====")

    if load_agent:
        agent_d = cumulative_agent_df[cumulative_agent_df['algorithm'] == algo]
        print(f"\tAverage Distance Travelled: {agent_d['distance_traveled'].mean():.2f}")
        print(f"\tDistance Travelled Std Dev: {agent_d['distance_traveled'].std():.2f}")
        print(f"\t--")
        print(f"\tAverage Battery Used (kWh): {agent_d['battery_used'].mean():.2f}")
        print(f"\tBattery Used Std Dev (kWh): {agent_d['battery_used'].std():.2f}")
        print(f"\t--")
        print(f"\tAverage Reward: {agent_d['reward'].mean():.2f}")
        print(f"\tReward Std Dev: {agent_d['reward'].std():.2f}")
        print(f"\t--")
    if load_station:
        station_d = cumulative_station_df[cumulative_station_df['algorithm'] == algo]
        print(f"\tMax Traffic: {station_d['traffic'].max():.2f}")
        #print(f"\tTraffic Std Dev: {station_d['traffic'].std():.2f}")

if load_agent:
    print("\n\n")
    for season in cumulative_agent_df['season'].unique():
        season_data = cumulative_agent_df[cumulative_agent_df['season'] == season]
        
        for algo in season_data['algorithm'].unique():
            algo_data = season_data[season_data['algorithm'] == algo]
    
            avg_energy_used = algo_data['battery_used'].mean()
            std_energy_used = algo_data['battery_used'].std()
    
            print(f"Algorotim: {algo} - Season: {season} \
            - Avg. Energy {avg_energy_used:.2f} - Std. Energy {std_energy_used:.2f}")

===== Algorithm: DQN =====
	Average Distance Travelled: 37.15
	Distance Travelled Std Dev: 1.59
	--
	Average Battery Used (kWh): 19363.11
	Battery Used Std Dev (kWh): 4869.33
	--
	Average Reward: -99.85
	Reward Std Dev: 13.74
	--
	Max Traffic: 23.00
===== Algorithm: CMA =====
	Average Distance Travelled: 36.89
	Distance Travelled Std Dev: 1.39
	--
	Average Battery Used (kWh): 19738.98
	Battery Used Std Dev (kWh): 4806.36
	--
	Average Reward: -107.84
	Reward Std Dev: 13.19
	--
	Max Traffic: 28.00



Algorotim: DQN - Season: winter             - Avg. Energy 15803.10 - Std. Energy 3322.84
Algorotim: CMA - Season: winter             - Avg. Energy 16294.41 - Std. Energy 3378.06
Algorotim: DQN - Season: summer             - Avg. Energy 22923.12 - Std. Energy 3321.28
Algorotim: CMA - Season: summer             - Avg. Energy 23183.56 - Std. Energy 3325.69
